In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# Read the csv file
df = pd.read_csv('data.csv', header=None)

# Remove empty rows
df = df.replace(np.nan,0)

# Convert the dataframe to a numpy array
data = df.to_numpy()

In [2]:
data

array([[0.0, 'R', 'P', ..., 'R', 'R', 'R']], dtype=object)

In [3]:
data.shape

(1, 200000)

In [4]:
Mapping = {'R':1,'P':2,'S':3}

def map_values(x):
    return Mapping.get(x, x)

In [5]:
vectorized_map_values = np.vectorize(map_values)
converted_data = vectorized_map_values(data)

In [6]:
converted_data.dtype

dtype('float64')

In [7]:
segments = []

for i in range(0, converted_data.shape[1] - 20):
    segment = converted_data[0][i:i+20]
    segments.append(segment)


In [35]:
segments

AttributeError: 'list' object has no attribute 'shape'

In [9]:
dataset = tf.data.Dataset.from_tensor_slices(converted_data)

In [10]:
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(200000,), dtype=tf.float64, name=None)>

In [11]:
BATCH_SIZE = 20
batched_data= dataset.batch(BATCH_SIZE, drop_remainder=True)

In [31]:
batched_data


<BatchDataset element_spec=TensorSpec(shape=(20, 200000), dtype=tf.float64, name=None)>

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

VOCAB_SIZE = 4
EMBEDDING_DIM = 32
RNN_UNITS = 1024

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (20, None, 4)             16        
                                                                 
 lstm_1 (LSTM)               (20, None, 1024)          4214784   
                                                                 
 dense_1 (Dense)             (20, None, 4)             4100      
                                                                 
Total params: 4,218,900
Trainable params: 4,218,900
Non-trainable params: 0
_________________________________________________________________


In [37]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [39]:
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x00000191D554F9D0>>

In [38]:
model.fit(segments,epochs = 10) #next attempt check what datasets can go into this function, make sure I am puttin gthe right stuff

KeyboardInterrupt: 

In [32]:
model.input_shape

(20, None)

AttributeError: 'BatchDataset' object has no attribute 'reshape'